In [1]:
# Cofiguration
# from MasterThesis.core import config
# config.config_notebook(False)

# Load libraries
from typing import List, Callable, Union, Tuple
import multiprocessing as mp
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os
import matplotlib.patches as mpatches
import numpy as np
import torch
from tqdm.autonotebook import tqdm

# custom libary
import MasterThesis.preprocessing as DP
from MasterThesis.EDA import image_label_sanity_check
from MasterThesis import EDA
from MasterThesis.utils.classification import wandb_utils

# from MasterThesis.utils import metrics, simclr, data_for_seg
# from MasterThesis.utils import data_for_seg

import wandb
wandb.login(key="ed6b4f7a25cd803c9ce7a66dbfba2353fe5bb5d2")

/home/omarcastano/.pyenv/versions/3.7.13/envs/MasterThesis/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: omar_castno (omar_castano). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/omarcastano/.netrc


True

In [2]:
drive_path = '/mnt/h/Mi unidad'

In [3]:
select_classes = ["airplane", "automobile", "bird"]

In [4]:
#Load metadata
path_to_metadata =  f'/mnt/h/Mi unidad/Maestria/Theses/CIFAR/metadata.csv'
metadata = pd.read_csv(path_to_metadata).sample(frac=1, random_state=42)
metadata = metadata.query(f"Classes in  {select_classes}")
print(metadata.shape)
metadata.head()

(18000, 3)


,Image,Labels,Classes
12628,automobile/12628.npz,1,automobile
39991,bird/39991.npz,2,bird
14871,automobile/14871.npz,1,automobile
21819,bird/21819.npz,2,bird
5026,bird/5026.npz,2,bird


In [10]:
project = "CLF_CIFAR3"
entity = "omar_castano"
name = "report_cifar"
version = [
    "RandomInit",
    "SimCLR",
    "RandomInit",
    "SimCLR",
    # "RandomInit",
]
train_size = [
    0.005,
    0.005,
    0.01,
    0.01,
    # 0.8,
]


bar_plots = {}
for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
    plot = wandb_utils.barplot_metrics_from_wandb(
        wandb,
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
    )

    bar_plots.update({metric: plot})

In [11]:
bar_plots["F1_score"]

In [8]:
def standar_report(wandb, project, entity, name, version, train_size):

    bar_plots = {}
    for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
        plot = wandb_utils.barplot_metrics_from_wandb(
            wandb,
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        bar_plots.update({metric: plot})

    line_plots = {}
    for metric in ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
        plot = wandb_utils.lineplot_metrics_from_wandb(
            wandb,
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        line_plots.update({metric: plot})

    table = wandb_utils.get_table(
        wandb,
        project=project,
        entity=entity,
        version=version,
        train_size=train_size,
    )

    pr_curve = {}
    for label in select_classes:

        plot = wandb_utils.plot_pr_curve(table.query(f"label=='{label}'"), color="train_size", line_dash="version")

        pr_curve.update({label: plot})

    loss_table = wandb_utils.get_table(
        wandb,
        project=project,
        entity=entity,
        version=version,
        train_size=train_size,
        table_name="Loss",
    )

    loss_plot = wandb_utils.plot_loss_curves(loss_table, color="train_size", line_dash="version")

    with wandb.init(
        project=project,
        entity=entity,
        config={"version": "report", "train_size": None},
        name=name,
    ) as run:

        run.log({"Label Distribution":px.histogram(metadata, x = "Classes", histnorm="percent")})
        [run.log({f"Bareplot {metric}": plot}) for metric, plot in bar_plots.items()]
        [run.log({f"Lineplot {metric}": plot}) for metric, plot in line_plots.items()]
        [run.log({f"PR Curve {label}": plot}) for label, plot in pr_curve.items()]
        run.log({"Loss Plots": loss_plot})

In [9]:
standar_report(
    wandb,
    project="CLF_cifar",
    entity="omar_castano",
    name="report_randominit_cifar",
    version=["RandomInit", "RandomInit", "RandomInit", "RandomInit"],
    train_size=[0.01, 0.05, 0.1, 0.8],
)

-------------------------------------------------------------------------------------
The number of runs found with train_size=0.01 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.05 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.1 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.8 and version=RandomInit is 5
-------------------------------------------------------------------------------------


-------------------------------------------------------------------------------------
The number of runs found with train_size=0.01 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.05 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.1 and version=RandomInit is 5
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
The number of runs found with train_size=0.8 and version=RandomInit is 5
-------------------------------------------------------------------------------------
